In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

from sklearn.metrics import root_mean_squared_error, mean_squared_error

In [27]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcap/02-Experiment-tracking/mlruns/1', creation_time=1716566344394, experiment_id='1', last_update_time=1716566344394, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [6]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [7]:
df_train = read_dataframe("./data/yellow_tripdata_2023-01.parquet")
df_val = read_dataframe("./data/yellow_tripdata_2023-02.parquet")

In [8]:
len(df_train), len(df_val)

(3009173, 2855951)

In [9]:
# df_train.duration.describe

In [10]:
df_train["PU_DO"] = df_train["PULocationID"] + '_' + df_train["PULocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + '_' + df_val["PULocationID"]

In [11]:
df_val ;

In [12]:
categorical = ['PULocationID','DOLocationID'] #["PU_DO"]
#numerical = ['trip_distance']

dv = DictVectorizer()

# train_dicts = df_train[categorical + numerical].to_dict(orient = "records")
train_dicts = df_train[categorical].to_dict(orient = "records")
X_train = dv.fit_transform(train_dicts)


In [13]:
# val_dicts = df_val[categorical + numerical].to_dict(orient = "records")
val_dicts = df_val[categorical].to_dict(orient = "records")
X_val = dv.transform(val_dicts)

In [14]:
target= 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [15]:
X_val[0].toarray().shape

(1, 515)

In [16]:
X_train[0].toarray().shape
# X_train[0].toarray()

(1, 515)

In [17]:
# testing model using jan data, and using feb data for validation(testing)
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)


In [18]:
root_mean_squared_error(y_val, y_pred)

7.811817745843695

In [19]:
with open('./models/lin_reg.bin', 'wb' ) as f_out :
    pickle.dump((dv, lr), f_out)

In [20]:
# testing model with its own traning data as validation data
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
root_mean_squared_error(y_train, y_pred)

7.649261934850555

In [28]:
# testing model with its traning data as validation data
with mlflow.start_run():

    mlflow.set_tag("developer", "Ganesh")

    mlflow.log_param("train-data-path", "./data/yellow_tripdata_2023-01.parquet")
    mlflow.log_param("valid-data-path", "./data/yellow_tripdata_2023-02.parquet")
    
    alpha = 0.001
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)